In [2]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import plotly.graph_objects as go
import numpy as np
from multiprocessing import Pool, cpu_count
from functools import lru_cache

In [3]:


reform = Reform.from_dict({
  "gov.contrib.congress.delauro.american_family_act.baby_bonus": {
    "2024-01-01.2100-12-31": 2400
  },
  "gov.irs.credits.ctc.amount.arpa[0].amount": {
    "2023-01-01.2028-12-31": 3600
  },
  "gov.irs.credits.ctc.amount.arpa[1].amount": {
    "2023-01-01.2028-12-31": 3000
  },
  "gov.irs.credits.ctc.phase_out.arpa.in_effect": {
    "2023-01-01.2028-12-31": True
  },
  "gov.irs.credits.ctc.refundable.fully_refundable": {
    "2023-01-01.2028-12-31": True
  }
}, country_id="us")




In [4]:
# Initialize simulations
baseline = Microsimulation(dataset="enhanced_cps_2022")
reformed = Microsimulation(reform=reform, dataset="enhanced_cps_2022")

In [5]:
@lru_cache(maxsize=None)
def calculate_difference(year):
    try:
        baseline_person = baseline.calculate("household_net_income", period=year, map_to="person")
        reformed_person = reformed.calculate("household_net_income", period=year, map_to="person")
        difference = (reformed_person - baseline_person).sum()
        return year, difference
    except Exception as e:
        print(f"Error calculating for year {year}: {str(e)}")
        return year, None

In [6]:
def main():
    years = list(range(2025, 2035))
    
    # Calculate differences sequentially
    results = [calculate_difference(year) for year in years]
    
    # Filter out any None results from errors
    valid_results = [(year, diff) for year, diff in results if diff is not None]
    valid_years, differences = zip(*valid_results)

    # Create Plotly bar chart
    fig = go.Figure(data=[
        go.Bar(name='Impact', x=valid_years, y=differences)
    ])

    # Update layout
    fig.update_layout(
        title='Impact of CTC Reform on Household Net Income (2025-2034)',
        xaxis_title='Year',
        yaxis_title='Total Difference in Household Net Income',
        yaxis_tickformat='$,.0f',
    )

    # Add hover template to show values in billions
    fig.update_traces(
        hovertemplate='Year: %{x}<br>Impact: $%{y:,.0f}<br>($%{y:,.1f} billion)',
        text=[f'${d/1e9:.1f}B' for d in differences],
        textposition='outside'
    )

    # Show the plot
    fig.show()

    # Print numerical results
    for year, difference in zip(valid_years, differences):
        print(f"{year}: ${difference/1e9:.2f} billion")

: 

In [7]:
if __name__ == "__main__":
    main()